In [1]:
%pip install youtube-search


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from youtube_search import YoutubeSearch

videos = YoutubeSearch("25-26시즌 NBA", max_results=5).to_dict()
videos

[{'id': 'M7c8iRpdOx0',
  'thumbnails': ['https://i.ytimg.com/vi/M7c8iRpdOx0/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLA_Lru5ECNGXHDeDImgi_zMRo5sNw',
   'https://i.ytimg.com/vi/M7c8iRpdOx0/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDLao6B10q8BdOff8-rtnu9csNSBw'],
  'title': '25-26시즌 프리뷰｜NBA 최악의 팀이 품은 달. 너무너무너무 궁금합니다',
  'long_desc': None,
  'channel': '조코피TV',
  'duration': '14:51',
  'views': '조회수 17,680회',
  'publish_time': '1개월 전',
  'url_suffix': '/watch?v=M7c8iRpdOx0&pp=ygUPMjUtMjbsi5zspowgTkJB0gcJCfwJAYcqIYzv'},
 {'id': 'a3Oqseuip4U',
  'thumbnails': ['https://i.ytimg.com/vi/a3Oqseuip4U/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCgeko_WkB80A21NgQE3Ahkl1V-9Q',
   'https://i.ytimg.com/vi/a3Oqseuip4U/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLC6BVtvKuWYpogPthfVhIUENtTvwA'],
  'title': "'ESPN 버전' 3가지 티어로 살펴보는 2025-26시즌 최종 순위 ",
  'long_desc': None,
  'channel': '조코피TV',

In [3]:
video_url = 'https://youtube.com' + videos[0]['url_suffix']
video_url

'https://youtube.com/watch?v=M7c8iRpdOx0&pp=ygUPMjUtMjbsi5zspowgTkJB0gcJCfwJAYcqIYzv'

In [4]:
%pip install youtube-transcript-api --upgrade


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# YouTubeTranscriptApi 1.2.3 올바른 사용법
from youtube_transcript_api import YouTubeTranscriptApi


def get_video_id(video_url):
  return video_url.split('watch?v=')[1].split('&')[0]


# video_id 추출
video_id = get_video_id(video_url)
print(f"Video ID: {video_id}")

# API 인스턴스 생성 및 자막 가져오기
api = YouTubeTranscriptApi()
fetched_transcript = api.fetch(video_id, languages=['ko', 'en'])

# raw 데이터로 변환
transcript_datas = fetched_transcript.to_raw_data()

transcript_datas

Video ID: M7c8iRpdOx0


IpBlocked: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=M7c8iRpdOx0! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

Ways to work around this are explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-ipblocked-exception).


If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide th

In [51]:
from langchain_core.documents import Document

for v in videos:
  fetched_scripts = api.fetch(v['id'], languages=['ko', 'en'])

  text = ' '.join(fetched_script.text for fetched_script in fetched_scripts)

  v['content'] = Document(
      page_content=' '.join(fetched_script.text for fetched_script in fetched_scripts),
      metadata={"source": v['id']}
  )

videos

IpBlocked: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=M7c8iRpdOx0! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

Ways to work around this are explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-ipblocked-exception).


If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide th

In [52]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages([
  ("system", "다음 영상에 대한 요약을 한국어로 만들어줘 : \n\n{context}")
])

chain = create_stuff_documents_chain(llm, prompt)

result = chain.invoke({"context": videos[0]['content']})
result

KeyError: 'content'

In [ ]:
from tqdm import tqdm

for v in tqdm(videos):
  v['summary'] = chain.invoke({"context": v['content']})

videos